# ToyaSapo 問い合わせデータ分析（統合版）

## 🚀 自動化のポイント
- **サービスアカウントキー**: Google Driveから自動読み込み
- **Azureキー**: Google Driveから自動読み込み
- **メールデータ**: Firestoreから直接取得（CSVアップロード不要）

## 🔧 初回セットアップ（1回だけ）
1. Google Driveの `001_PBL_チームD_AI4th` フォルダに以下を配置
   - `toyasapo-firebase-adminsdk-fbsvc-93683c6f93.json`
   - `azure_textanalytics_key.txt`

## 📝 毎回の使い方
1. このノートブックを開く
2. 「ランタイム」→「すべてのセルを実行」
3. Google Driveのアクセス許可を承認
4. 完了！

## 💾 保存されるデータ
`analysis_results/latest` に以下が保存されます：
- `updated_at`: 更新日時
- `total_count`: 総件数
- `keywords_top20`: キーワードTOP20
- `keywords_tfidf`: 重要キーワード（TF-IDF）
- `bigrams_top20`: N-gram（2語の組み合わせ）
- `cooccurrence`: 共起分析
- `disease`: 病名・診断
- `symptoms`: 症状
- `concerns`: 悩み・状況
- `inquiry_types`: 相談内容
- `sentiment_summary`: 感情分析（positive/negative/neutral/mixed）

## 1. セットアップ（ライブラリとDrive接続）

In [ ]:
# 必要なライブラリをインストール
!apt-get -y install fonts-ipafont-gothic > /dev/null 2>&1
!pip install janome azure-ai-textanalytics japanize-matplotlib firebase-admin -q

print("✅ インストール完了")

In [ ]:
# ライブラリのインポート
import pandas as pd
import matplotlib.pyplot as plt
import japanize_matplotlib
from collections import Counter, defaultdict
from janome.tokenizer import Tokenizer
from datetime import datetime
import warnings
import os
import json
import numpy as np
from statistics import mean

# Azure & Firebase
import firebase_admin
from firebase_admin import credentials, firestore
from azure.ai.textanalytics import TextAnalyticsClient
from azure.core.credentials import AzureKeyCredential

warnings.filterwarnings('ignore')

# 日本語フォント設定
plt.rcParams['font.family'] = 'IPAPGothic'
japanize_matplotlib.japanize()
plt.rcParams['figure.figsize'] = (12, 6)

print("✅ ライブラリ読み込み完了")

In [ ]:
# Google Driveをマウント
from google.colab import drive
drive.mount('/content/drive')

print("✅ Google Drive マウント完了")

## 2. 認証情報とFirebase初期化

In [ ]:
# --- 設定 ---
BASE_PATH = '/content/drive/MyDrive/001_PBL_チームD_AI4th'
FIREBASE_KEY_PATH = os.path.join(BASE_PATH, 'toyasapo-firebase-adminsdk-fbsvc-93683c6f93.json')
AZURE_KEY_PATH = os.path.join(BASE_PATH, 'azure_textanalytics_key.txt')
AZURE_ENDPOINT = "https://toyasapo-language.cognitiveservices.azure.com/"

# 1. Firebase初期化
if not os.path.exists(FIREBASE_KEY_PATH):
    print("❌ エラー: firebase-key.json が見つかりません")
    print(f"   パス: {FIREBASE_KEY_PATH}")
else:
    if not firebase_admin._apps:
        try:
            cred = credentials.Certificate(FIREBASE_KEY_PATH)
            firebase_admin.initialize_app(cred)
            print("✅ Firebase初期化完了")
        except Exception as e:
            print(f"❌ Firebase初期化エラー: {e}")
    else:
        print("✅ Firebase既に初期化済み")
    
    db = firestore.client()
    print("✅ Firestore接続完了")

# 2. Azure Key読み込み
azure_key = None
if os.path.exists(AZURE_KEY_PATH):
    try:
        with open(AZURE_KEY_PATH, 'r') as f:
            azure_key = f.read().strip()
        print("✅ Azure APIキー読み込み完了")
    except Exception as e:
        print(f"❌ Azure Key読み込みエラー: {e}")
else:
    print("❌ エラー: azure_textanalytics_key.txt が見つかりません")

## 3. メールデータ取得

In [ ]:
print("📧 Firestoreからメールデータを取得中...")

emails_ref = db.collection('emails')
docs = emails_ref.stream()

data = []
for doc in docs:
    doc_data = doc.to_dict()
    if not doc_data.get('isDeleted', False):
        data.append(doc_data)

df = pd.DataFrame(data)

print(f"✅ データ取得完了: {len(df)}件")

# カラム特定
content_col = None
for col in ['inquiry', 'content', 'body', '本文', '本文(匿名化済)', 'Content', 'Body', 'message']:
    if col in df.columns:
        content_col = col
        break

print(f"本文カラム: {content_col}")

## 4. 形態素解析・TF-IDF・N-gram分析

In [ ]:
tokenizer = Tokenizer()

stop_words = {
    'こと', 'もの', 'ため', 'よう', 'ところ', 'とき', '方', '人', '私', '自分',
    '今', '後', '前', '中', '上', '下', '件', '点', '旨', '等', '頃', '際',
    '度', '分', '回', '月', '日', '年', '時', '週', '先', '次', '別', '他',
    '以上', '以下', '以前', '以降', '場合', '状態', '状況', '理由', '原因',
    '確認', '質問', '相談', '依頼', '希望', '要望', '対応', '返答',
    '宛名', '冒頭', '文末', '本文', '件名', '内容', '下記', '上記',
    '可能', '必要', '大変', '承知', '了解', '申し訳', '恐れ入り',
    'こちら', 'そちら', '時間', '紹介', '変更',
    'あちら', 'どちら',
    'メール', 'アドレス', '問い合わせ', '問合せ', '連絡', '返信', '送信',
    '宛先', 'お問い合わせ', 'フォーム', 'メールアドレス',
    'お世話', 'お願い', 'ご連絡', 'ご確認', 'ご質問', 'ご相談',
    'はじめまして', '初めまして', 'よろしく', '宜しく', '皆様', '皆さま',
    'toyano', 'mental', 'TMC', 'toya', 'とやの', 'メンタル', 'クリニック',
    'niigata', '新潟',
    'facebook', 'twitter', 'instagram', 'LINE', 'line', 'http', 'https', 'www',
    '名前', '削除', '担当', '者', '様', '氏', '殿', '御中', 'さん', '先生',
    '電話', '番号', '住所', '現在', '最近',
}

def extract_nouns(text):
    if pd.isna(text):
        return []
    nouns = []
    for token in tokenizer.tokenize(str(text)):
        if token.part_of_speech.startswith('名詞'):
            word = token.surface
            if (len(word) >= 2 and
                word not in stop_words and
                not word.isascii() and
                not word.isdigit() and
                not any(c in word for c in '[](){}「」【】、。・>=</:\"')):
                nouns.append(word)
    return nouns

print("形態素解析中...")
all_nouns = []
if content_col:
    for text in df[content_col]:
        all_nouns.extend(extract_nouns(text))
    noun_counts = Counter(all_nouns)
    print(f"✅ 完了: {len(all_nouns)}個の名詞を抽出")
else:
    print("⚠️ 本文カラムが見つかりません")
    noun_counts = Counter()

In [ ]:
# TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer

def tokenize_to_nouns(text):
    nouns = extract_nouns(text)
    return ' '.join(nouns)

tfidf_ranking = []
if content_col and len(df) > 0:
    texts = df[content_col].fillna('').apply(tokenize_to_nouns).tolist()
    vectorizer = TfidfVectorizer(max_features=100)
    try:
        tfidf_matrix = vectorizer.fit_transform(texts)
        feature_names = vectorizer.get_feature_names_out()
        mean_tfidf = np.array(tfidf_matrix.mean(axis=0)).flatten()
        tfidf_ranking = sorted(zip(feature_names, mean_tfidf), key=lambda x: x[1], reverse=True)
        print("✅ TF-IDF計算完了")
    except ValueError:
        print("⚠️ データ不足のためTF-IDFスキップ")

In [ ]:
# N-gram
def extract_bigrams(text):
    nouns = extract_nouns(text)
    if len(nouns) < 2:
        return []
    bigrams = []
    for i in range(len(nouns) - 1):
        if nouns[i] != nouns[i+1]:
            bigram = f"{nouns[i]}_{nouns[i+1]}"
            bigrams.append(bigram)
    return bigrams

all_bigrams = []
if content_col:
    for text in df[content_col]:
        all_bigrams.extend(extract_bigrams(text))
    bigram_counts = Counter(all_bigrams)
    print("✅ N-gram抽出完了")
else:
    bigram_counts = Counter()

In [ ]:
# 共起分析
def get_cooccurrence(texts, target_word):
    cooccur = Counter()
    for text in texts:
        nouns = extract_nouns(text)
        if target_word in nouns:
            for noun in nouns:
                if noun != target_word:
                    cooccur[noun] += 1
    return cooccur

target_words = ['うつ', '発達', '予約', '不安', '仕事', '休職']
cooccurrence_results = {}

if content_col:
    texts = df[content_col]
    for target in target_words:
        cooccur = get_cooccurrence(texts, target)
        cooccurrence_results[target] = cooccur.most_common(10)
    print(f"✅ 共起分析完了 ({len(target_words)}語)")

## 5. カテゴリ別分析（医療・メンタルヘルス）

In [ ]:
keywords_dict = {
    '病名・診断': [
        'うつ病', 'うつ', '鬱', '鬱病', '抑うつ', '大うつ',
        '双極性障害', '双極性', '躁うつ', '躁鬱', '気分障害', '躁状態',
        'パニック障害', 'パニック', '社交不安', '社会不安', '対人恐怖', '広場恐怖', '全般性不安',
        '発達障害', 'ADHD', '注意欠陥', 'ASD', '自閉症', 'アスペルガー', '学習障害', 'LD',
        '適応障害', 'PTSD', '心的外傷', 'トラウマ', '急性ストレス',
        '不眠症', '睡眠障害', '過眠', '睡眠時無呼吸',
        '強迫性障害', '強迫症', 'OCD', '解離性障害', '解離',
        '摂食障害', '過食', '拒食', '過食症', '拒食症',
        '依存症', 'アルコール依存', 'ゲーム依存', 'ギャンブル依存',
        '人格障害', 'パーソナリティ障害', 'BPD', '境界性',
        '統合失調症', '認知症', '更年期障害', '自律神経失調症',
        'PMS', 'PMDD', 'HSP', 'SAD', '季節性'
    ],
    '症状': [
        '眠れない', '不眠', '寝れない', '早朝覚醒', '中途覚醒', '過眠', '悪夢',
        '不安', '動悸', '息苦しい', '過呼吸', '緊張', '恐怖', 'パニック発作',
        '落ち込み', '憂うつ', 'やる気が出ない', '無気力', '虚しい', '希望がない',
        '頭痛', 'めまい', '吐き気', '食欲不振', '食欲がない', '倦怠感', 'だるい', '疲れ',
        '手の震え', '動悸', '胸が苦しい', '息切れ',
        '集中できない', '物忘れ', '頭が回らない', '決断できない',
        '涙が止まらない', '泣いてしまう', 'イライラ', '怒り', '感情の起伏',
        '人が怖い', '外出できない', '引きこもり',
        '希死念慮', '死にたい', '自傷', '消えたい', 'リストカット',
        'フラッシュバック', '幻聴', '幻覚', '妄想', '離人感'
    ],
    '悩み・状況': [
        '休職', '復職', '退職', '仕事', '職場', '上司', '同僚', '部下', '転職',
        'パワハラ', 'セクハラ', 'モラハラ', 'いじめ', '人間関係', '残業', '過労',
        '家族', '親', '母', '父', '夫', '妻', '子供', '息子', '娘',
        '育児', '介護', '離婚', '結婚', '嫁姑', '義母', '義父',
        '恋愛', '恋人', '彼氏', '彼女', '友人', '孤独', '孤立',
        '学校', '不登校', '受験', '進路', '就活', '留年', '退学',
        'ストレス', '引きこもり', '借金', '経済的', 'DV', '虐待'
    ],
    '相談内容': [
        '予約', 'キャンセル', '変更', '初診', '再診', '通院',
        '診断書', '紹介状', '意見書', '証明書',
        'セカンドオピニオン', '薬', '処方', '副作用', '減薬', '断薬',
        'カウンセリング', '心理検査', '知能検査', 'WAIS', '認知行動療法', 'CBT',
        '費用', '料金', '保険', '自立支援', '障害年金', '傷病手当', '生活保護',
        '障害者手帳', '手帳',
        '入院', '転院', '紹介', 'オンライン', 'リモート'
    ]
}

synonyms = {
    'うつ': ['うつ', 'うつ病', '鬱', '鬱病', '抑うつ', '大うつ'],
    '双極性障害': ['双極性障害', '双極性', '躁うつ', '躁鬱', '気分障害'],
    'パニック障害': ['パニック障害', 'パニック', 'パニック発作'],
    '発達障害': ['発達障害', 'ADHD', '注意欠陥', 'ASD', '自閉症', 'アスペルガー'],
    '不眠': ['不眠', '不眠症', '眠れない', '寝れない', '睡眠障害', '早朝覚醒', '中途覚醒'],
    '不安': ['不安', '不安感', '心配'],
    '適応障害': ['適応障害'],
    'PTSD': ['PTSD', '心的外傷', 'トラウマ'],
    '強迫性障害': ['強迫性障害', '強迫症', 'OCD'],
    '摂食障害': ['摂食障害', '過食', '拒食', '過食症', '拒食症'],
}

def count_keywords_with_synonyms(texts, category_keywords, synonyms_dict):
    counts = Counter()
    if texts.empty:
        return counts
        
    for text in texts:
        if pd.isna(text):
            continue
        text = str(text)
        
        counted_groups = set()
        for group_name, words in synonyms_dict.items():
            if group_name in counted_groups:
                continue
            for word in words:
                if word in text and word in category_keywords:
                    counts[group_name] += 1
                    counted_groups.add(group_name)
                    break
        
        for keyword in category_keywords:
            in_synonym = any(keyword in words for words in synonyms_dict.values())
            if not in_synonym and keyword in text:
                counts[keyword] += 1
    return counts

results_unified = {}
if content_col and len(df) > 0:
    for category, keywords in keywords_dict.items():
        counts = count_keywords_with_synonyms(df[content_col], keywords, synonyms)
        results_unified[category] = counts.most_common(10)
    print("✅ カテゴリ別分析完了")
else:
    for category in keywords_dict.keys():
        results_unified[category] = []
    print("⚠️ データなしのためカテゴリ分析スキップ")

## 6. Azure Text Analytics 感情分析
※Azure APIキーが設定されている場合のみ実行されます

In [ ]:
sentiment_summary_result = None

if azure_key and content_col:
    print("🧠 感情分析を開始します（Azure AI Language）...")
    
    # 分析対象の準備（文字数制限などを考慮）
    credential = AzureKeyCredential(azure_key)
    text_analytics_client = TextAnalyticsClient(endpoint=AZURE_ENDPOINT, credential=credential)
    
    # バッチ処理用のリスト作成
    docs_to_analyze = []
    # Azure制限: 1リクエスト最大10件、1ドキュメント最大5120文字
    for text in df[content_col]:
        if pd.isna(text) or str(text).strip() == "":
            continue
        docs_to_analyze.append(str(text)[:5000])

    print(f"   分析対象: {len(docs_to_analyze)}件")
    
    counts = {'positive': 0, 'negative': 0, 'neutral': 0, 'mixed': 0}
    scores_sum = {'positive': [], 'negative': [], 'neutral': []}
    
    batch_size = 10
    processed_count = 0
    
    for i in range(0, len(docs_to_analyze), batch_size):
        batch = docs_to_analyze[i:i + batch_size]
        try:
            response = text_analytics_client.analyze_sentiment(batch, language="ja")
            for result in response:
                if not result.is_error:
                    counts[result.sentiment] += 1
                    scores_sum['positive'].append(result.confidence_scores.positive)
                    scores_sum['negative'].append(result.confidence_scores.negative)
                    scores_sum['neutral'].append(result.confidence_scores.neutral)
        except Exception as e:
            print(f"   ⚠️ Batch error at {i}: {e}")
            
        processed_count += len(batch)
        if processed_count % 100 == 0:
            print(f"   ... {processed_count}件 完了")
            
    # 平均スコア
    avg_scores = {
        'positive': mean(scores_sum['positive']) if scores_sum['positive'] else 0,
        'negative': mean(scores_sum['negative']) if scores_sum['negative'] else 0,
        'neutral': mean(scores_sum['neutral']) if scores_sum['neutral'] else 0
    }
    
    sentiment_summary_result = {
        'positive_count': counts['positive'],
        'negative_count': counts['negative'],
        'neutral_count': counts['neutral'],
        'mixed_count': counts['mixed'],
        'average_scores': avg_scores
    }
    print("✅ 感情分析完了")
    print(sentiment_summary_result)

else:
    print("⚠️ Azure APIキーがない、またはデータがないため感情分析をスキップします")
    sentiment_summary_result = None

## 7. Firestoreに保存

In [ ]:
# 保存データの構築
analysis_data = {
    'updated_at': datetime.now().isoformat(),
    'total_count': len(df),

    # キーワード分析結果
    'keywords_top20': [{'word': w, 'count': c} for w, c in noun_counts.most_common(20)],
    'keywords_tfidf': [{'word': w, 'score': round(float(s), 4)} for w, s in tfidf_ranking[:20]],
    'bigrams_top20': [{'words': b.replace('_', ' → '), 'count': c} for b, c in bigram_counts.most_common(20)],
    'cooccurrence': {
        target: [{'word': w, 'count': c} for w, c in words]
        for target, words in cooccurrence_results.items()
    },

    # カテゴリ分析結果
    'disease': [{'word': w, 'count': c} for w, c in results_unified['病名・診断'][:10]],
    'symptoms': [{'word': w, 'count': c} for w, c in results_unified['症状'][:10]],
    'concerns': [{'word': w, 'count': c} for w, c in results_unified['悩み・状況'][:10]],
    'inquiry_types': [{'word': w, 'count': c} for w, c in results_unified['相談内容'][:10]],
}

# 感情分析結果があれば追加
if sentiment_summary_result:
    analysis_data['sentiment_summary'] = sentiment_summary_result

# Firestoreに保存
doc_ref = db.collection('analysis_results').document('latest')
doc_ref.set(analysis_data, merge=True)

print("✅ すべての分析結果をFirestoreに保存しました！")
print(f"\n保存先: analysis_results/latest")
print(f"更新日時: {analysis_data['updated_at']}")
if sentiment_summary_result:
    print("✨ 感情分析データも含まれています")